### Missing data is handled in different ways - 
* CCA (Complete Case Analysis) - also called as 'List-wise deletion', removes the entire row if any column has missing value, but this approach is not followed since the important information can be lost on removing the entire row
* Simple Imputer - For Univariate variables - 
    * For missing numerical values, we can replace with mean, median or end of distribution
    * For the missing categorical variables, we can replace through 'mode' or 'missing(writing missing in place of the missing values)'
    * Filling the missing values with any of the arbitrary values (any random values)
* For multivariate - 
    * KNN imputer
    * Iterative imputer
    * MICE
        * So there are three different ways of handling the missing values in a column.

* CCA - Assumptions- The first assumption that is followed is that the missing data should be random, and not the first 50 rows or last 50 rows in continuation.
* The data has to be random so that the important features are not missed
* Also called as MCAR (Missing Completely at Random).

Disadvantages of CCA - 
* Important information is lost.
* When using our models in production, the model will not know how to handle the missing data.
* It can exclude a large fraction of original data set

Advantages of CCA - 
* Easy to apply

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('data_science_job.csv')

In [3]:
df.head(2)

enrollee_id      city  city_development_index gender  \
0         8949  city_103                   0.920   Male   
1        29725   city_40                   0.776   Male   

       relevent_experience enrolled_university education_level  \
0  Has relevent experience       no_enrollment        Graduate   
1   No relevent experience       no_enrollment        Graduate   

  major_discipline  experience company_size company_type  training_hours  \
0             STEM        20.0          NaN          NaN            36.0   
1             STEM        15.0        50-99      Pvt Ltd            47.0   

   target  
0     1.0  
1     0.0

In [4]:
df.isnull().sum()

enrollee_id                  0
city                         0
city_development_index     479
gender                    4508
relevent_experience          0
enrolled_university        386
education_level            460
major_discipline          2813
experience                  65
company_size              5938
company_type              6140
training_hours             766
target                       0
dtype: int64

* this data set contains the missing values

In [5]:
df.shape

(19158, 13)

In [6]:
len(df.index)

19158

In [7]:
df.columns

Index(['enrollee_id', 'city', 'city_development_index', 'gender',
       'relevent_experience', 'enrolled_university', 'education_level',
       'major_discipline', 'experience', 'company_size', 'company_type',
       'training_hours', 'target'],
      dtype='object')

In [9]:
df.columns.value_counts().sum()

13

In [10]:
df.columns.value_counts()

enrollee_id               1
city                      1
city_development_index    1
gender                    1
relevent_experience       1
enrolled_university       1
education_level           1
major_discipline          1
experience                1
company_size              1
company_type              1
training_hours            1
target                    1
dtype: int64

In [12]:
df.isnull().mean()*100

enrollee_id                0.000000
city                       0.000000
city_development_index     2.500261
gender                    23.530640
relevent_experience        0.000000
enrolled_university        2.014824
education_level            2.401086
major_discipline          14.683161
experience                 0.339284
company_size              30.994885
company_type              32.049274
training_hours             3.998330
target                     0.000000
dtype: float64

* we will apply CCA only where the total percentage of the missing data is less than 5% and not more than that

In [13]:
## filtering out the columns whose mean >0.05 (5%) and then applying CCA on those columns - 

In [18]:
cols = [var for var in df.columns if df[var].isnull().mean() < 0.05 and df[var].isnull().mean()>0]

In [19]:
cols

['city_development_index',
 'enrolled_university',
 'education_level',
 'experience',
 'training_hours']

In [21]:
df[cols].sample(5)

city_development_index enrolled_university education_level  experience  \
1219                    0.920    Full time course        Graduate        10.0   
9701                    0.555       no_enrollment        Graduate         1.0   
10670                   0.624       no_enrollment        Graduate         5.0   
10541                   0.920    Full time course        Graduate        19.0   
6644                    0.555       no_enrollment        Graduate         2.0   

       training_hours  
1219            278.0  
9701             26.0  
10670            24.0  
10541           292.0  
6644             17.0

In [25]:
len(df[cols].dropna()) / len(df)
## so here 89% columns will remain after removing the missing columns whose mean is < 0.05(5%)

0.8968577095730244

In [23]:
df1 = df[cols].dropna()

In [24]:
df1.shape

(17182, 5)

In [28]:
df['enrolled_university'].value_counts(dropna=False)

no_enrollment       13817
Full time course     3757
Part time course     1198
NaN                   386
Name: enrolled_university, dtype: int64

* so there are 386 missing values in the 'enrolled university' column

In [29]:
len(df)

19158

In [30]:
len(df['enrolled_university'])

19158

In [31]:
len(df1['enrolled_university'])

17182

In [39]:
compare = pd.concat([df['enrolled_university'].value_counts() / len(df) , 
                     df1['enrolled_university'].value_counts() / len(df1)], 
                    axis = 1)

In [40]:
compare.columns = ['original dataset', 'new dataset']

In [41]:
compare

original dataset  new dataset
no_enrollment             0.721213     0.735188
Full time course          0.196106     0.200733
Part time course          0.062533     0.064079

In [42]:
df['enrolled_university'].value_counts()

no_enrollment       13817
Full time course     3757
Part time course     1198
Name: enrolled_university, dtype: int64

In [43]:
df['enrolled_university'].value_counts().sum()

18772

In [44]:
df1['enrolled_university'].value_counts().sum()

17182

In [50]:
df.head(4)

enrollee_id      city  city_development_index gender  \
0         8949  city_103                   0.920   Male   
1        29725   city_40                   0.776   Male   
2        11561   city_21                   0.624    NaN   
3        33241  city_115                   0.789    NaN   

       relevent_experience enrolled_university education_level  \
0  Has relevent experience       no_enrollment        Graduate   
1   No relevent experience       no_enrollment        Graduate   
2   No relevent experience    Full time course        Graduate   
3   No relevent experience                 NaN        Graduate   

  major_discipline  experience company_size company_type  training_hours  \
0             STEM        20.0          NaN          NaN            36.0   
1             STEM        15.0        50-99      Pvt Ltd            47.0   
2             STEM         5.0          NaN          NaN            83.0   
3  Business Degree         0.0          NaN      Pvt Ltd            52.0   

   target  
0     1.0  
1     0.0  
2     0.0  
3     1.0

In [49]:
(df['target']==1.0).value_counts()

False    14381
True      4777
Name: target, dtype: int64

In [51]:
(df['target']==0.0).sum()

14381

In [52]:
(df['target']==1.0).sum()

4777

In [53]:
(df['major_discipline']== 'STEM').sum()

14492

In [54]:
(df['major_discipline']=='STEM').value_counts()

True     14492
False     4666
Name: major_discipline, dtype: int64